In [5]:
THIS REALLY DOES NOT WORK!!!!

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter

# Parameters
Gmax = 40      # mT/m
Smax = 150     # mT/m/ms
M0 = 10        # mT·ms/m
dt = 0.01      # ms
Tmax = 50      # ms
N = int(Tmax / dt)
t = np.linspace(0, Tmax, N)

# --- Initial random but valid waveform ---
np.random.seed(42)
G = np.random.randn(N)
G -= np.mean(G)
G[0] = 0
G[-1] = 0

# Enforce slew and gradient constraints
for _ in range(50):
    dG = np.diff(G) / dt
    dG = np.clip(dG, -Smax, Smax)
#    G[1:] = np.cumsum(np.insert(dG, 0, 0)) * dt
    G = np.cumsum(np.insert(dG, 0, 0)) * dt
    G = np.clip(G, -Gmax, Gmax)
    G[0] = 0
    G[-1] = 0

# Rescale to correct moment
G *= M0 / (np.sum(G) * dt)

gradient_history = [G.copy()]
slew_history = [np.gradient(G, dt)]

# --- Trapezoidal target (for reference/validation) ---
def trapezoidal_solution(M0, Gmax, Smax, dt):
    ramp_time = Gmax / Smax
    ramp_area = 0.5 * Gmax * ramp_time
    if 2 * ramp_area >= M0:
        ramp_time = np.sqrt(M0 / Smax)
        Gpeak = Smax * ramp_time
        flat_time = 0
    else:
        flat_area = M0 - 2 * ramp_area
        flat_time = flat_area / Gmax
        Gpeak = Gmax
    total_time = 2 * ramp_time + flat_time
    Nt = int(total_time / dt)
    t_opt = np.linspace(0, total_time, Nt)
    G_opt = np.zeros(Nt)
    for i, ti in enumerate(t_opt):
        if ti < ramp_time:
            G_opt[i] = Smax * ti
        elif ti < ramp_time + flat_time:
            G_opt[i] = Gpeak
        else:
            G_opt[i] = Gpeak - Smax * (ti - ramp_time - flat_time)
    return G_opt, t_opt

G_target, t_target = trapezoidal_solution(M0, Gmax, Smax, dt)

# --- Optimization loop ---
G_iter = G.copy()
num_iters = 80

for iter_num in range(num_iters):
    # Compress time a bit
    Tmax_new = Tmax * (0.985 ** iter_num)
    t_new = np.linspace(0, Tmax_new, N)
    G_interp = np.interp(t_new, t, G_iter)

    # Enforce slew rate
    dG = np.diff(G_interp) / dt
    dG = np.clip(dG, -Smax, Smax)
    G_new = np.cumsum(np.insert(dG, 0, 0)) * dt
    G_new = np.clip(G_new, -Gmax, Gmax)

    # Moment correction
    G_new -= np.mean(G_new)
    G_new *= M0 / (np.sum(G_new) * dt)
    G_new[0] = 0
    G_new[-1] = 0

    G_iter = G_new
    gradient_history.append(G_iter.copy())
    slew_history.append(np.gradient(G_iter, dt))

# --- Animation ---
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(6, 5), sharex=True)

line_g, = ax1.plot([], [], 'b-', lw=2)
line_s, = ax2.plot([], [], 'r-', lw=2)

ax1.set_ylabel("Gradient (mT/m)")
ax2.set_ylabel("Slew Rate (mT/m/ms)")
ax2.set_xlabel("Time (ms)")

ax1.set_ylim(-Gmax * 1.2, Gmax * 1.2)
ax2.set_ylim(-Smax * 1.2, Smax * 1.2)
ax1.grid()
ax2.grid()
ax1.set_title("Gradient and Slew Rate Optimization")

def update(frame):
    g = gradient_history[frame]
    s = slew_history[frame]
    t_anim = np.linspace(0, len(g) * dt, len(g))
    line_g.set_data(t_anim, g)
    line_s.set_data(t_anim, s)
    return line_g, line_s

ani = FuncAnimation(fig, update, frames=len(gradient_history), blit=True, interval=100)

# Save to animated gif
ani.save("gradient_optimization_with_slew.gif", writer=PillowWriter(fps=10))
print("Saved: gradient_optimization_with_slew.gif")


SyntaxError: invalid syntax (393325800.py, line 1)